# Work with data assets in R

This notebook shows how to read and write data assets into your Machine Learning workspace from R.  

## Prerequisites

To use this notebook, you'll need to set up a few things.

1.  Make sure the **R** kernel is selected in the top right of the notebook toolbar.
2.  In your compute instance toolbar, install `azureml-fsspec` into the `azureml_py310_sdkv2` kernel:

    ```
    conda activate azureml_py310_sdkv2
    pip install azureml-fsspec
    conda deactivate
    ```

In [ ]:
# Check version of reticulate.  If not at least 1.26, use a newer compute instance.
packageVersion('reticulate')

## Load reticulate and set condaenv

This R script will use the `reticulate` package, which runs Python code from within R.  

First, load the package and set the conda environment to be used for the Python code:

In [ ]:
library(reticulate)
use_condaenv("azureml_py310_sdkv2")
print("Environment is set")

## Establish connection with workspace
This next bit of code in Python will configure the connection to your workspace.  

In [ ]:
py_code <- "from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential=credential)"

py_run_string(py_code)
print("ml_client is configured")

## Create a data asset

This code shows creating a data asset from an online blob storage.  You can als use local baths, ADLS gen2, or datastore paths.  For more details, see [Create data assets](https://learn.microsoft.com/azure/machine-learning/how-to-create-data-assets?tabs=cli#create-a-uri_file-data-asset).

In [ ]:
# Creata a data asset from online path
py_code <- "from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_path = 'https://azuremlexamples.blob.core.windows.net/datasets/credit_card/default_of_credit_card_clients.csv'
my_name = 'credit-card'
my_description = 'credit card data'

my_data = Data(
    path=my_path,
    type=AssetTypes.URI_FILE,
    description=my_description,
    name=my_name
)

ml_client.data.create_or_update(my_data)"

py_run_string(py_code)
print(paste("Created data asset:", py$my_name ))

# Retrieve a data asset

You can retrieve any data asset in your workspace.  All you need is the name and version number of the asset.  

In [ ]:
# Replace <MY_NAME> and <MY_VERSION> with your values
py_code <- "my_name = '<MY_NAME>'
my_version = '<MY_VERSION>'
data_asset = ml_client.data.get(name=my_name, version=my_version)
data_uri = data_asset.path"

For example, use this code to retrieve the data asset you created above:  

In [ ]:
py_code <- "my_name = 'credit-card'
my_version = '1'
data_asset = ml_client.data.get(name=my_name, version=my_version)
data_uri = data_asset.path"

Now run the code and print the URI

In [ ]:
py_run_string(py_code)
print(paste("URI path is", py$data_uri))

Use the URI path to read your data asset into R:

In [ ]:
pd <- import("pandas")
cc <- pd$read_csv(py$data_uri)
head(cc)